In [14]:
from IPython.display import display_html


def restart_kernel():
    display_html("<script>Jupyter.notebook.kernel.restart()</script>", raw=True)


restart_kernel()

In [15]:
import os

# Set environment variable for TensorFlow
# os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"

import tensorflow as tf

# Configure GPU memory growth
gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        # Set memory growth before initializing GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices("GPU")
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Handle potential errors here
        print(e)

In [16]:
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import numpy as np
import cv2
import random
import os

import os
import cv2
import numpy as np
import pandas as pd
from PIL import Image

import pandas as pd
import numpy as np
import cv2
from skimage import exposure
from scipy.ndimage import gaussian_filter
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import LearningRateScheduler
import math
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import pandas as pd
import cv2
from skimage import exposure
from scipy.ndimage import gaussian_filter
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from tensorflow.keras.optimizers import SGD

In [17]:
import numpy as np
import random
import tensorflow as tf
import torch

# Set seeds to ensure reproducibility
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)  # For multi-GPU setups

## Model Below

In [18]:
# load the data
X_train = np.load("../../aws_s3/messidor_224/X_train.npy")
y_train = np.load("../../aws_s3/messidor_224/y_train.npy")
X_test = np.load("../../aws_s3/messidor_224/X_test.npy")
y_test = np.load("../../aws_s3/messidor_224/y_test.npy")

In [19]:
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)

In [20]:
base_model = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_tensor=None,
    input_shape=(224, 224, 3),
    pooling="avg",
)

> freeze the base model

In [21]:
base_model.trainable = False

> Create a new model on top with dropout

In [22]:
num_classes = 4
epochs = 50
# Define the inputs
inputs = tf.keras.Input(shape=(224, 224, 3))

# Ensure the base_model is running in inference mode.
x = base_model(inputs, training=False)

x = layers.Dropout(0.5)(x)  # Dropout layer with 50% dropout rate
# Adding FC (Fully Connected) layers
x = layers.Dense(1000)(x)
x = layers.Dropout(0.5)(x)  # Another Dropout layer with 50% dropout rate
# Adding FC (Fully Connected) layers
x = layers.Dense(500)(x)

# Adding a final layer with SoftMax activation for classification
outputs = layers.Dense(num_classes, activation="softmax")(x)

# Creating the model
model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.summary(show_trainable=True)

Model: "model_1"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         Y          
                                                                            
 efficientnetb0 (Functional)  (None, 1280)             4049571   N          
                                                                            
 dropout_2 (Dropout)         (None, 1280)              0         Y          
                                                                            
 dense_3 (Dense)             (None, 1000)              1281000   Y          
                                                                            
 dropout_3 (Dropout)         (None, 1000)              0         Y          
                                                                            
 dense_4 (Dense)             (None, 500)               5005

> Train the model on new data

In [23]:
# learning rate scheduler
base_learning_rate = 1e-1
max_learning_rate = 1e-2


# creating a learning rate scheduler
def lr_scheduler(epoch, lr):
    if epoch < 15:
        return lr
    elif epoch < 24:
        return lr - 0.01
    elif epoch < 35:
        return lr
    else:
        return lr * tf.math.exp(-0.1)


# Create an instance of SGD optimizer with initial learning rate
optimizer = SGD(learning_rate=base_learning_rate, momentum=0.9, clipnorm=1.0)


# Function to compute class weights using TensorFlow
def compute_class_weights(labels):
    # Convert labels to a 1D tensor if not already
    labels = tf.reshape(labels, [-1])

    # Get unique classes and their indices and counts
    unique_classes, _, class_counts = tf.unique_with_counts(labels)

    # Compute total number of samples
    total_samples = tf.reduce_sum(class_counts)

    # Compute class weights
    class_weights = total_samples / (len(unique_classes) * class_counts)

    # Create a class weights dictionary mapping class indices to their respective weights
    class_weight_dict = dict(zip(unique_classes.numpy(), class_weights.numpy()))

    return class_weight_dict


class_weight_dict = compute_class_weights(y_train)

# When fitting the model, include the learning rate scheduler callback
lr_scheduler = LearningRateScheduler(lr_scheduler)

In [24]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

(TensorShape([1163, 224, 224, 3]),
 TensorShape([1163]),
 TensorShape([24, 224, 224, 3]),
 TensorShape([24]))

In [25]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))

Num GPUs Available:  0


In [26]:
model.compile(
    optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

print("Fitting the top layer of the model")
model.fit(
    X_train,
    y_train,
    epochs=epochs,
    batch_size=32,
    class_weight=class_weight_dict,
    callbacks=[lr_scheduler],
)

Fitting the top layer of the model
Epoch 1/50


37/37 [==============================] - 23s 457ms/step - loss: 2.5894 - accuracy: 0.3138 - lr: 0.1000
Epoch 2/50
37/37 [==============================] - 17s 451ms/step - loss: 2.8601 - accuracy: 0.3044 - lr: 0.1000
Epoch 3/50
37/37 [==============================] - 17s 449ms/step - loss: 2.7349 - accuracy: 0.3207 - lr: 0.1000
Epoch 4/50
37/37 [==============================] - 17s 449ms/step - loss: 3.0149 - accuracy: 0.3113 - lr: 0.1000
Epoch 5/50
37/37 [==============================] - 17s 449ms/step - loss: 2.7447 - accuracy: 0.3233 - lr: 0.1000
Epoch 6/50
37/37 [==============================] - 17s 449ms/step - loss: 3.0540 - accuracy: 0.3061 - lr: 0.1000
Epoch 7/50
37/37 [==============================] - 17s 454ms/step - loss: 2.6421 - accuracy: 0.3302 - lr: 0.1000
Epoch 8/50
37/37 [==============================] - 17s 452ms/step - loss: 3.1791 - accuracy: 0.3216 - lr: 0.1000
Epoch 9/50
37/37 [==============================] - 17s 446ms/step - loss: 3.0804 - accuracy: 0.329

> Fine tuning: unfreeze all or part of the base model and retrain the whole model end-to-end with a very low learning rate.

In [27]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Unfreeze the base model
base_model.trainable = True
model.summary(show_trainable=True)

# Re-instantiate the optimizer
optimizer = SGD(learning_rate=base_learning_rate, momentum=0.9, clipnorm=1.0)

model.compile(
    optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

print("Fitting the end-to-end model")
# Train end-to-end. Stop before overfit
model.fit(
    X_train,
    y_train,
    epochs=epochs,
    batch_size=32,
    class_weight=class_weight_dict,
    callbacks=[lr_scheduler],
)

Model: "model_1"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         Y          
                                                                            
 efficientnetb0 (Functional)  (None, 1280)             4049571   Y          
                                                                            
 dropout_2 (Dropout)         (None, 1280)              0         Y          
                                                                            
 dense_3 (Dense)             (None, 1000)              1281000   Y          
                                                                            
 dropout_3 (Dropout)         (None, 1000)              0         Y          
                                                                            
 dense_4 (Dense)             (None, 500)               5005

> Evaluate on test data

In [ ]:
# model.save("my_model.keras")

In [ ]:
print("Test dataset evaluation")
model.evaluate(X_test, y_test)

Test dataset evaluation


1/1 [==============================] - 2s 2s/step - loss: 2.9535 - accuracy: 0.6250


[2.953517198562622, 0.625]